In [1]:
import zipfile
zip = zipfile.ZipFile('fra-eng.zip')
zip.extractall()

In [2]:
import string,re
from unicodedata import normalize
from numpy import array,argmax
from pickle import load,dump
from numpy.random import rand,shuffle

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM,Dense,Embedding,RepeatVector,TimeDistributed
from nltk.translate.bleu_score import SmoothingFunction,corpus_bleu
smoothie = SmoothingFunction().method4

In [4]:
def load_file(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [5]:
# split a loaded document into sentences
def splitting_sentence(doc):
	sentences = doc.strip().split('\n')
	pairs = [sentence.split('\t') for sentence in  sentences]
	return pairs

In [6]:
def clean_pairs(sentences):
    cleaned = list()
 
    #preparing regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))

# preparing translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)

  # iterating over each pair
    for pair in sentences:
        clean_pair = list()
  
        for sentence in pair:
# normalizing unicode characters
            sentence = normalize('NFD', sentence).encode('ascii', 'ignore')
            sentence = sentence.decode('UTF-8')
# tokenizing on white space
            sentence = sentence.split()
# converting to lowercase
            sentence = [word.lower() for word in sentence]
# removing punctuation from each token
            sentence = [word.translate(table) for word in sentence]
# removing non-printable chars form each token
            sentence = [re_print.sub('', w) for w in sentence]
            
            sentence = [word for word in sentence if word.isalpha()]
# storing as string
            clean_pair.append(' '.join(sentence))
        cleaned.append(clean_pair)
    return array(cleaned)


In [7]:
def saving_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print(filename,': Saved')

In [8]:
# load dataset

filename = 'fra.txt'
doc = load_file(filename)

# split into english-french pairs
pairs = splitting_sentence(doc)

# clean sentences
clean_pairs = clean_pairs(pairs)

# save clean pairs to file
saving_clean_data(clean_pairs, 'english-french.pkl')

print('English','-->',"French")
# spot check
for i in range(25):
    print(clean_pairs[i,0],'-->',clean_pairs[i,1])

english-french.pkl : Saved
English --> French
go --> va
go --> marche
go --> bouge
hi --> salut
hi --> salut
run --> cours
run --> courez
run --> prenez vos jambes a vos cous
run --> file
run --> filez
run --> cours
run --> fuyez
run --> fuyons
run --> cours
run --> courez
run --> prenez vos jambes a vos cous
run --> file
run --> filez
run --> cours
run --> fuyez
run --> fuyons
who --> qui
wow --> ca alors
wow --> waouh
wow --> wah


In [9]:
# load a clean dataset
def loading_cleaned_data(filename):
    return load(open(filename, 'rb'))

In [10]:
# load dataset
data = loading_cleaned_data('english-french.pkl')
print(data.shape) 

(194513, 3)


In [11]:
# reducing dataset size (scaling) 

new_data_size = 20000
dataset = data[:new_data_size, :]

# randomly shuffling the dataset to get proper training and testing data
shuffle(dataset)

# splitting into training and testing (90%-10%)
train, test = dataset[:18000], dataset[18000:]

# saving the cleaned data,train data and test data 
saving_clean_data(dataset, 'english-french-both.pkl')
saving_clean_data(train, 'english-french-train.pkl')
saving_clean_data(test, 'english-french-test.pkl')

english-french-both.pkl : Saved
english-french-train.pkl : Saved
english-french-test.pkl : Saved


In [12]:
dataset = loading_cleaned_data('english-french-both.pkl')
train = loading_cleaned_data('english-french-train.pkl')
test = loading_cleaned_data('english-french-test.pkl')

In [13]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)

In [14]:
#english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])

print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

#french tokenizer

fra_tokenizer = create_tokenizer(dataset[:, 1])
fra_vocab_size = len(fra_tokenizer.word_index) + 1
fra_length = max_length(dataset[:, 1])
print('French Vocabulary Size: %d' % fra_vocab_size)
print('French Max Length: %d' % (fra_length))

English Vocabulary Size: 3414
English Max Length: 5
French Vocabulary Size: 6988
French Max Length: 11


In [15]:
# Input and Output sequence must be encoded to integers and padded to the maximum phrase length
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	x = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	x = pad_sequences(x, maxlen=length, padding='post')
	return x

# One hot encoding to max phrase length
def one_hot_encoding(sequences, vocab_size):
	y_1 = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		y_1.append(encoded)
	y = array(y_1)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [16]:
# preparing training data
trainX = encode_sequences(fra_tokenizer, fra_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = one_hot_encoding(trainY, eng_vocab_size)

# prepare testing data
testX = encode_sequences(fra_tokenizer, fra_length, test[:, 1])
testY = encode_sequences(eng_tokenizer,eng_length, test[:, 0])
testY = one_hot_encoding(testY, eng_vocab_size)

In [17]:
print('training size:',trainX.shape,trainY.shape)
print('testing size:',testX.shape,testY.shape)

training size: (18000, 11) (18000, 5, 3414)
testing size: (2000, 11) (2000, 5, 3414)


In [18]:
def model_building(source_vocab, target_vocab, source_len, target_len, units):
	model = Sequential()
	model.add(Embedding(source_vocab, units, input_length=source_len, mask_zero=True))
	model.add(LSTM(units))
	model.add(RepeatVector(target_len))
	model.add(LSTM(units, return_sequences=True))
	model.add(TimeDistributed(Dense(target_vocab, activation='softmax')))
	return model

In [19]:
model = model_building(fra_vocab_size, eng_vocab_size, fra_length, eng_length, 512)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

In [20]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 512)           3577856   
                                                                 
 lstm (LSTM)                 (None, 512)               2099200   
                                                                 
 repeat_vector (RepeatVector  (None, 5, 512)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 5, 512)            2099200   
                                                                 
 time_distributed (TimeDistr  (None, 5, 3414)          1751382   
 ibuted)                                                         
                                                                 
Total params: 9,527,638
Trainable params: 9,527,638
Non-

In [21]:
# Stop model if accuracy of the model doesn't changes by more than 0.01 
# Patience = 5 : After each 5 epochs if no improvement is there then training will be stopped.
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_acc',patience= 5,min_delta=0.01)

In [22]:
# fit model
model.fit(trainX, trainY, epochs= 50, batch_size=25, validation_data=(testX, testY), verbose=2,callbacks=[es])

Epoch 1/20
720/720 - 266s - loss: 3.5927 - acc: 0.4664 - val_loss: 3.0713 - val_acc: 0.5190 - 266s/epoch - 369ms/step
Epoch 2/20
720/720 - 251s - loss: 2.7210 - acc: 0.5537 - val_loss: 2.5952 - val_acc: 0.5803 - 251s/epoch - 348ms/step
Epoch 3/20
720/720 - 239s - loss: 2.1729 - acc: 0.6111 - val_loss: 2.2558 - val_acc: 0.6193 - 239s/epoch - 331ms/step
Epoch 4/20
720/720 - 242s - loss: 1.7368 - acc: 0.6594 - val_loss: 2.0429 - val_acc: 0.6453 - 242s/epoch - 336ms/step
Epoch 5/20
720/720 - 235s - loss: 1.3636 - acc: 0.7077 - val_loss: 1.8961 - val_acc: 0.6663 - 235s/epoch - 327ms/step
Epoch 6/20
720/720 - 235s - loss: 1.0453 - acc: 0.7588 - val_loss: 1.7852 - val_acc: 0.6912 - 235s/epoch - 327ms/step
Epoch 7/20
720/720 - 236s - loss: 0.7888 - acc: 0.8063 - val_loss: 1.7221 - val_acc: 0.7074 - 236s/epoch - 327ms/step
Epoch 8/20
720/720 - 236s - loss: 0.5916 - acc: 0.8479 - val_loss: 1.6956 - val_acc: 0.7148 - 236s/epoch - 327ms/step
Epoch 9/20
720/720 - 266s - loss: 0.4500 - acc: 0.8809 -

In [23]:
# mapping integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:   
			return word
	return None

In [24]:
# generating target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [25]:
def evaluate_model(model, tokenizer, sources, raw_dataset):
  
  # Creating empty lists for actual phrases(French) and predicted phrases(English) 
  actual,predicted = list(),list()
  a,b,c = list(),list(),list()
  for i,source in enumerate(sources):

    # reshaping to the required size
    source = source.reshape((1, source.shape[0]))

    # predicting for the english tokenizer
    translation = predict_sequence(model, eng_tokenizer, source)
    # raw_dataset = raw_dataset[i].split(' ') 
    # print(raw_dataset[i][1])

    raw_src,raw_target = raw_dataset[i][1],raw_dataset[i][0]
    
    # First 10 Predictions
    if i <= 10:
      print('source = ',raw_src,'<--->', ' target = ',raw_target,'<--->','  predicted = ',translation)

    actual.append([raw_target.split()])
    predicted.append(translation.split())
  
  # calculating BLEU score
  print('-------------------------------------------')
  print('BLEU Score :')
  print('BLEU score-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0),smoothing_function=smoothie,auto_reweigh=False))
  print('BLEU score-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie,auto_reweigh=False))
  print('BLEU score-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0),smoothing_function=smoothie,auto_reweigh=False))
  print('BLEU score-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie,auto_reweigh=False))

In [26]:
evaluate_model(model,eng_tokenizer,trainX,train)

source =  tom etait sincere <--->  target =  tom was sincere <--->   predicted =  tom meant sincere
source =  tom etait tres motive <--->  target =  tom was excited <--->   predicted =  tom was excited
source =  ca se passera bien pour vous <--->  target =  youll be fine <--->   predicted =  youll be fine
source =  essaie de resister <--->  target =  try to resist <--->   predicted =  try to resist
source =  depechetoi de venir <--->  target =  come quickly <--->   predicted =  come quickly
source =  tu es un amour <--->  target =  youre a doll <--->   predicted =  youre a doll
source =  ils vous ont vu <--->  target =  they saw you <--->   predicted =  they saw you
source =  quittez tom <--->  target =  leave tom <--->   predicted =  leave tom
source =  il me faut de la nourriture <--->  target =  i need food <--->   predicted =  i need food
source =  etesvous catholique <--->  target =  are you catholic <--->   predicted =  are you catholic
source =  je ne ronfle pas <--->  target = 

In [27]:
evaluate_model(model, eng_tokenizer, testX, test)

source =  il a abandonne tout espoir <--->  target =  he gave up hope <--->   predicted =  he gave up hope
source =  vous captez <--->  target =  do you get it <--->   predicted =  you you it
source =  estce quelle a aime <--->  target =  did she like it <--->   predicted =  is that for it
source =  reste assise <--->  target =  stay seated <--->   predicted =  sit tight
source =  ce nest pas un poisson <--->  target =  it isnt a fish <--->   predicted =  its not a fish
source =  je vous ai suivie <--->  target =  i followed you <--->   predicted =  i followed you
source =  vous etes intrepide <--->  target =  youre fearless <--->   predicted =  youre fearless
source =  dirigezvous vers lest <--->  target =  head east <--->   predicted =  head east
source =  cest ici que nous sommes <--->  target =  here we are <--->   predicted =  its wide here
source =  tu es mechante <--->  target =  youre mean <--->   predicted =  youre wicked
source =  jai gere <--->  target =  i handled it <---> 